In [1]:
import os
import trimesh
import random
import torch
from tqdm import tqdm
import pickle

from scripts.utils import save_mesh_with_color_clinical, save_mesh_with_color_acdc, dir_checker
from scripts import postprocessing

import numpy as np
from sklearn.utils import shuffle
from sklearn import svm
import gc

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


In [2]:
model_dir = './models/no_nn/'
for m in tqdm(os.listdir(model_dir)):
    path_out = './predictions/inference'
    dir_in = './data_for_inference/obj/'
    name = '045.obj'
    
    with open(model_dir + m, 'rb') as f:
        model = pickle.load(f)

    preprocessed_dir = 'TL_prepared_data/sdf'
    preprocessed_dir += '/' + name[:-4]
    
    mesh = trimesh.load(dir_in+name)

    sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
    sdf = sdf.view(len(sdf), -1)
    
    colors = torch.load(f'{preprocessed_dir}/color.pt')
    colors= torch.argmax(colors, dim=1)

    predicted = model.predict(sdf.float())
    
    name = name[:-4] + '_' + m
    
    dirname = dir_checker(path_out, name)
    save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
    save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')
    
    mesh = trimesh.load(f'{dirname}/{name[:-4]}_predicted.obj')
    
    postprocessing(mesh, f'{dirname}/{name[:-4]}_postprocessed.obj')



  0%|                                                    | 0/10 [00:00<?, ?it/s]

Folder for predictions: ./predictions/inference/045_gaussian


 10%|████▍                                       | 1/10 [00:01<00:09,  1.03s/it]

Folder for predictions: ./predictions/inference/045_linear_svc


 20%|████████▊                                   | 2/10 [00:01<00:06,  1.19it/s]

...

 80%|███████████████████████████████████▏        | 8/10 [03:40<00:27, 13.53s/it]

Folder for predictions: ./predictions/inference/045_lda


 90%|███████████████████████████████████████▌    | 9/10 [03:41<00:09,  9.54s/it]

Folder for predictions: ./predictions/inference/045_random_forest


100%|███████████████████████████████████████████| 10/10 [03:41<00:00, 22.19s/it]


In [1]:

import os
import trimesh
import random
import torch
from tqdm import tqdm
import pickle

from scripts.utils import save_mesh_with_color_clinical, save_mesh_with_color_acdc, dir_checker
from scripts import postprocessing

import numpy as np
from sklearn.utils import shuffle
from sklearn import svm
import gc



from scripts.models import Segmentation_Network_full
from scripts.models import resnet1, resnet18, resnet34, resnet50, resnet101, resnet152
from scripts.models import UNet
import torch

In [3]:
device = 'cuda:0'

In [5]:
model_dir = './models/from_cluster'
for m in tqdm(os.listdir(model_dir)):
    path_out = './predictions/inference'
    dir_in = './data_for_inference/obj/'
    name = '047.obj'
    
    model = torch.load(f'{model_dir}/{m}/best.pt').to(device)

    preprocessed_dir = 'TL_prepared_data/sdf'
    preprocessed_dir += '/' + name[:-4]
    
    mesh = trimesh.load(dir_in+name)

    sdf = torch.load(f'{preprocessed_dir}/sdf.pt').float()
    #sdf = sdf.view(len(sdf), -1)
    
    colors = torch.load(f'{preprocessed_dir}/color.pt')
    colors= torch.argmax(colors, dim=1)

    sdfs = sdf.split(200)
    predicted = []
    for i in sdfs:
        predicted.append(torch.argmax(model(i.to(device)), dim=1).detach().cpu())

    model = None
    gc.collect()
    predicted = torch.concatenate(predicted, dim=0)
    
    name = name[:-4] + '_' + m[15:]
    
    dirname = dir_checker(path_out, name + '.obj')
    print(dirname)
    save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name}_original.obj')
    save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name}_predicted.obj')
    
    mesh = trimesh.load(f'{dirname}/{name}_predicted.obj')
    
    postprocessing(mesh, f'{dirname}/{name}_postprocessed.obj')


  0%|                                                     | 0/7 [00:00<?, ?it/s]

Folder for predictions: ./predictions/inference/047_resnet18
./predictions/inference/047_resnet18


 14%|██████▍                                      | 1/7 [00:01<00:07,  1.25s/it]

Folder for predictions: ./predictions/inference/047_resnet16
./predictions/inference/047_resnet16


 29%|████████████▊                                | 2/7 [00:02<00:05,  1.04s/it]

...

 71%|████████████████████████████████▏            | 5/7 [00:12<00:05,  2.78s/it]

Folder for predictions: ./predictions/inference/047_resnet101
./predictions/inference/047_resnet101


 86%|██████████████████████████████████████▌      | 6/7 [00:14<00:02,  2.55s/it]

Folder for predictions: ./predictions/inference/047_resnet152
./predictions/inference/047_resnet152


100%|█████████████████████████████████████████████| 7/7 [00:16<00:00,  2.41s/it]
